### 라이브러리

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [3]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [4]:
# 랭크변경상태
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [5]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [6]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [7]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

train['식사가능자수(석식)'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'] - train['본사출장자수'])
test['식사가능자수(석식)'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'] - test['본사출장자수'])

In [8]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [9]:
#중식
week1 = {
    52	:	52	,
    31	:	51	,
    33	:	50	,
    30	:	49	,
    32	:	48	,
    18	:	47	,
    46	:	46	,
    45	:	45	,
    47	:	44	,
    21	:	43	,
    44	:	42	,
    23	:	41	,
    51	:	40	,
    43	:	39	,
    28	:	38	,
    29	:	37	,
    42	:	36	,
    24	:	35	,
    17	:	34	,
    22	:	33	,
    26	:	32	,
    25	:	31	,
    27	:	30	,
    20	:	29	,
    15	:	28	,
    14	:	27	,
    16	:	26	,
    50	:	25	,
    19	:	24	,
    1	:	23	,
    3	:	22	,
    39	:	21	,
    36	:	20	,
    48	:	19	,
    2	:	18	,
    4	:	17	,
    37	:	16	,
    34	:	15	,
    13	:	14	,
    49	:	13	,
    35	:	12	,
    38	:	11	,
    12	:	10	,
    11	:	9	,
    6	:	8	,
    5	:	7	,
    9	:	6	,
    8	:	5	,
    10	:	4	,
    41	:	3	,
    7	:	2	,
    40	:	1	
}

#석식 위부터 작은수임

week2 = {
    52	:	52	,
    1	:	51	,
    31	:	50	,
    4	:	49	,
    48	:	48	,
    30	:	47	,
    26	:	46	,
    51	:	45	,
    43	:	44	,
    44	:	43	,
    46	:	42	,
    2	:	41	,
    17	:	40	,
    22	:	39	,
    33	:	38	,
    39	:	37	,
    29	:	36	,
    3	:	35	,
    32	:	34	,
    45	:	33	,
    18	:	32	,
    5	:	31	,
    47	:	30	,
    35	:	29	,
    23	:	28	,
    28	:	27	,
    37	:	26	,
    21	:	25	,
    19	:	24	,
    27	:	23	,
    24	:	22	,
    42	:	21	,
    9	:	20	,
    20	:	19	,
    38	:	18	,
    50	:	17	,
    25	:	16	,
    13	:	15	,
    16	:	14	,
    49	:	13	,
    12	:	12	,
    36	:	11	,
    15	:	10	,
    14	:	9	,
    34	:	8	,
    6	:	7	,
    11	:	6	,
    8	:	5	,
    7	:	4	,
    41	:	3	,
    10	:	2	,
    40	:	1	
}

train['주(중식)'] = train['주'].map(week1)
test['주(중식)'] = test['주'].map(week1)

train['주(석식)'] = train['주'].map(week2)
test['주(석식)'] = test['주'].map(week2)

train['주(중식)'] = train['주(중식)'].astype('int') # float형이라 int로 변경
test['주(중식)'] = test['주(중식)'].astype('int')
train['주(석식)'] = train['주(석식)'].astype('int')
test['주(석식)'] = test['주(석식)'].astype('int')

In [10]:
train['공휴일전후1'] = 0
test['공휴일전후1'] = 0

train['공휴일전후2'] = 0
test['공휴일전후2'] = 0

In [11]:
train['공휴일전후1'][4] = 1 #2
train['공휴일전후1'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후1'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후1'][152] = 1
train['공휴일전후1'][226] = 1
train['공휴일전후1'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후1'][224] = 1
train['공휴일전후1'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후1'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후1'][310] = 2
train['공휴일전후1'][311] = 1
train['공휴일전후1'][309] = 1
train['공휴일전후1'][330] = 1
train['공휴일전후1'][379] = 1
train['공휴일전후1'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후1'][470] = 1
train['공휴일전후1'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후1'][511] = 1
train['공휴일전후1'][565] = 1
train['공휴일전후1'][623] = 1
train['공휴일전후1'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후1'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후1'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후1'][815] = 2
train['공휴일전후1'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후1'][950] = 1
train['공휴일전후1'][951] = 1
train['공휴일전후1'][953] = 2
train['공휴일전후1'][954] = 1
train['공휴일전후1'][955] = 1
train['공휴일전후1'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후1'][1038] = 1
train['공휴일전후1'][1099] = 1
train['공휴일전후1'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후1'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후1'][10] =2
test['공휴일전후1'][20] = 1

In [12]:
train['공휴일전후2'][4] = 1 #2
train['공휴일전후2'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후2'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후2'][152] = 1
train['공휴일전후2'][226] = 1
train['공휴일전후2'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후2'][224] = 1
train['공휴일전후2'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후2'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후2'][310] = 2
train['공휴일전후2'][311] = 1
train['공휴일전후2'][309] = 1
train['공휴일전후2'][330] = 1
train['공휴일전후2'][379] = 1
train['공휴일전후2'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후2'][470] = 1
train['공휴일전후2'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후2'][511] = 1
train['공휴일전후2'][565] = 1
train['공휴일전후2'][623] = 1
train['공휴일전후2'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후2'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후2'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후2'][815] = 2
train['공휴일전후2'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후2'][950] = 1
train['공휴일전후2'][951] = 1
train['공휴일전후2'][953] = 2
train['공휴일전후2'][954] = 1
train['공휴일전후2'][955] = 1
train['공휴일전후2'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후2'][1038] = 1
train['공휴일전후2'][1099] = 1
train['공휴일전후2'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후2'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후2'][10] =2
test['공휴일전후2'][20] = 1





# test['공휴일전후2'][18] =2 # 석식에만 추가
# test['공휴일전후2'][42] =2 # 석식에만 추가

In [13]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후1'])
test = pd.get_dummies(test, columns=['공휴일전후1'])

train = pd.get_dummies(train, columns=['공휴일전후2'])
test = pd.get_dummies(test, columns=['공휴일전후2'])

In [14]:
test['공휴일전후1_0'][20] = 1
test['공휴일전후1_1'][20] = 0

test['공휴일전후2_0'][20] = 1
test['공휴일전후2_1'][20] = 0

In [15]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '식사가능자수(석식)', '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후1_0',
       '공휴일전후1_1', '공휴일전후1_2', '공휴일전후2_0', '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

In [16]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '요일(석식)', '년', '월', '일', '주', '식사가능자수', '식사가능자수(석식)',
       '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2',
       '공휴일전후2_0', '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

### TOP Score

In [17]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [20]:
# train = train[(train.석식계== 0) & (train.월 == 1)]
train = train.drop(train.index[[244,492,730,973]])



x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수(석식)','월(석식)', '일']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수(석식)', '월(석식)', '일']]

In [21]:
# 244
# 492
# 730
# 973

# 1월 자기개발만 삭제

In [22]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040276
0:	learn: 204.5783397	total: 137ms	remaining: 2m 17s
1:	learn: 199.2574371	total: 145ms	remaining: 1m 12s
2:	learn: 194.2908903	total: 149ms	remaining: 49.6s
3:	learn: 189.5642196	total: 153ms	remaining: 38.1s
4:	learn: 185.1211245	total: 157ms	remaining: 31.2s
5:	learn: 181.3558052	total: 158ms	remaining: 26.2s
6:	learn: 177.2369221	total: 161ms	remaining: 22.8s
7:	learn: 173.2171465	total: 164ms	remaining: 20.4s
8:	learn: 169.5265418	total: 168ms	remaining: 18.5s
9:	learn: 166.1230359	total: 170ms	remaining: 16.9s
10:	learn: 162.7955654	total: 174ms	remaining: 15.6s
11:	learn: 159.9276433	total: 176ms	remaining: 14.5s
12:	learn: 156.9358043	total: 178ms	remaining: 13.5s
13:	learn: 153.9569396	total: 180ms	remaining: 12.7s
14:	learn: 151.2316009	total: 182ms	remaining: 11.9s
15:	learn: 148.7122678	total: 184ms	remaining: 11.3s
16:	learn: 146.2605281	total: 186ms	remaining: 10.8s
17:	learn: 143.8918001	total: 189ms	remaining: 10.3s
18:	learn: 141.7208923	t

In [23]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,987.947075,148.371909
1,2021-01-28,953.878458,466.247357
2,2021-01-29,570.706346,203.705074


In [44]:
submission

,일자,중식계,석식계
0,2021-01-27,1034.335307,211.856858
1,2021-01-28,952.824874,492.803430
2,2021-01-29,590.069460,169.337479
3,2021-02-01,1235.656781,523.454714
4,2021-02-02,1054.141259,558.075415
5,2021-02-03,1066.575925,499.569888
6,2021-02-04,1019.942527,567.607799
7,2021-02-05,735.416766,298.016622
8,2021-02-08,1271.896751,697.864087
9,2021-02-09,1089.375602,592.683787


In [48]:
# submission.중식계 = submission.중식계-5

In [29]:
submission.to_csv('어출장.csv', index=False) # 1제거하고 중식돌린거.

In [28]:
submission.석식계.mean()

518.3798126659296